<a href="https://colab.research.google.com/github/sherna90/analisis_algoritmos/blob/master/tsp_demo/Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simulated Annealing Hyperaparameter Tuning

In [24]:
import numpy as np

from time import time
import scipy.stats as stats
from sklearn.utils.fixes import loguniform

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import train_test_split

# get some data
X, y = load_digits(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
# build a classifier
clf = SGDClassifier(loss='log',fit_intercept=True,shuffle=False)

In [25]:
clf.get_params()

{'alpha': 0.0001,
 'average': False,
 'class_weight': None,
 'early_stopping': False,
 'epsilon': 0.1,
 'eta0': 0.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'learning_rate': 'optimal',
 'loss': 'log',
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': 'l2',
 'power_t': 0.5,
 'random_state': None,
 'shuffle': False,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [30]:
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

0.9511784511784511

In [31]:
import scipy 

param_dist = {'penalty': ['l2','l1'],
              'eta0': loguniform(1e-4, 1e-2),
              'alpha': scipy.stats.expon(scale=.1)}

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

start = time()
random_search.fit(X_train, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=SGDClassifier(alpha=0.0001, average=False,
                                           class_weight=None,
                                           early_stopping=False, epsilon=0.1,
                                           eta0=0.0, fit_intercept=True,
                                           l1_ratio=0.15,
                                           learning_rate='optimal', loss='log',
                                           max_iter=1000, n_iter_no_change=5,
                                           n_jobs=None, penalty='l2',
                                           power_t=0.5, random_state=None,
                                           shuffle=False, tol=0.001,
                                           validation_fract...
                                           warm_start=False),
                   iid='deprecated', n_iter=20, n_jobs=None,
                   param_distributions={'alpha': <scip

In [32]:
random_search.best_params_

{'alpha': 0.3006454950643594, 'eta0': 0.00014617560407750082, 'penalty': 'l2'}

In [33]:
clf.set_params(**random_search.best_params_)

SGDClassifier(alpha=0.3006454950643594, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.00014617560407750082,
              fit_intercept=True, l1_ratio=0.15, learning_rate='optimal',
              loss='log', max_iter=1000, n_iter_no_change=5, n_jobs=None,
              penalty='l2', power_t=0.5, random_state=None, shuffle=False,
              tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [34]:
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.3006454950643594, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.00014617560407750082,
              fit_intercept=True, l1_ratio=0.15, learning_rate='optimal',
              loss='log', max_iter=1000, n_iter_no_change=5, n_jobs=None,
              penalty='l2', power_t=0.5, random_state=None, shuffle=False,
              tol=0.001, validation_fraction=0.1, verbose=0, warm_start=False)

In [35]:
clf.score(X_test,y_test)

0.9629629629629629

In [44]:
from sklearn.model_selection import ParameterSampler

def score(clf,X_train,y_train,X_test,y_test,par):
  clf.set_params(**par)
  clf.fit(X_train,y_train)
  return clf.score(X_test,y_test)

param_list = list(ParameterSampler(param_dist, n_iter=20))

max_val=-np.infty
min_val=np.infty
for i,par in enumerate(param_list):
  current_val=score(clf,X_train,y_train,X_test,y_test,par)
  min_val=min(current_val,min_val)
  max_val=max(current_val,max_val)
  print('Parametro {0}, score : {1}'.format(i,current_val))

print('min val {0}, max_val : {1}'.format(min_val,max_val))
 

Parametro 0, score : 0.9208754208754208
Parametro 1, score : 0.9208754208754208
Parametro 2, score : 0.9074074074074074
Parametro 3, score : 0.7239057239057239
Parametro 4, score : 0.8468013468013468
Parametro 5, score : 0.877104377104377
Parametro 6, score : 0.7676767676767676
Parametro 7, score : 0.9175084175084175
Parametro 8, score : 0.9595959595959596
Parametro 9, score : 0.877104377104377
Parametro 10, score : 0.9612794612794613
Parametro 11, score : 0.898989898989899
Parametro 12, score : 0.9074074074074074
Parametro 13, score : 0.7626262626262627
Parametro 14, score : 0.9545454545454546
Parametro 15, score : 0.7491582491582491
Parametro 16, score : 0.7861952861952862
Parametro 17, score : 0.9326599326599326
Parametro 18, score : 0.9191919191919192
Parametro 19, score : 0.9629629629629629
min val 0.7239057239057239, max_val : 0.9629629629629629


In [102]:
from sklearn.model_selection import ParameterSampler
import copy 

def mutation(param_dist,par,prior=False):
  new_par=copy.deepcopy(par) 
  param_to_update = np.random.choice(list(param_dist.keys()))
  param_vals = param_dist[param_to_update]
  if isinstance(param_vals,list):
    new_par[param_to_update]=np.random.choice(param_vals)
  else:
    if prior:
      new_par[param_to_update]=param_vals.rvs()
    else:
      new_par[param_to_update]=par[param_to_update]+np.abs(np.random.normal(0,1e-1))
  return new_par

param_list = list(ParameterSampler(param_dist, n_iter=1))

In [83]:
par=param_list[0]
print(par)
clf.set_params(**par)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

{'alpha': 0.14671042776547744, 'eta0': 0.004621528670708173, 'penalty': 'l2'}


0.9629629629629629

In [96]:
new_par=mutation(param_dist,par,prior=True)
print(new_par)
clf.set_params(**new_par)
clf.fit(X_train,y_train)
clf.score(X_test,y_test)

penalty
{'alpha': 0.14671042776547744, 'eta0': 0.004621528670708173, 'penalty': 'l1'}


0.8484848484848485

In [97]:
import math

class temperature_scheduling:
    
    def __init__(self,initial_temperature,min_temperature,gamma):
        self.initial_temperature=initial_temperature
        self.min_temperature=min_temperature
        self.temperature=initial_temperature
        self.gamma=gamma
        
    def exponential_decay(self):
        self.temperature=self.gamma*self.temperature
        return max(self.temperature,self.min_temperature)
    
    def fast_decay(self,k):
        self.temperature=self.initial_temperature/k
        return max(self.temperature,self.min_temperature)
    
    def logarithmic_decay(self,k):
        self.temperature=self.initial_temperature*math.log(2,2)/math.log(k+1,2)
        return max(self.temperature,self.min_temperature)

In [103]:
k=0.85
initial_temperature=1.2*(max_val-min_val)
min_temp=1e-4

par=param_list[0]
temp=temperature_scheduling(initial_temperature,min_temp,k)
current_val=score(clf,X_train,y_train,X_test,y_test,par)
epochs=10
loss=[]

for i in range(1,epochs):
    new_par=mutation(param_dist,par,prior=True)
    next_value=score(clf,X_train,y_train,X_test,y_test,new_par)
    delta = (current_val-next_value)
    T=temp.logarithmic_decay(i)
    if delta>0:
        par=copy.deepcopy(new_par)
    else:
        alpha=min(1,np.exp(delta/T))
        if alpha>np.random.random():
            par=copy.deepcopy(new_par)
            current_val=next_value
    loss.append(current_val)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_stochastic_gradient.py:557: ConvergenceWarning: Maximum number of iteration reached before convergence. Consider increasing max_iter to improve the fit.
  ConvergenceWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_mo

In [104]:
loss

[0.9494949494949495,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9629629629629629,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646464646,
 0.9646464646